In [10]:
import sys
import json
from os import getenv
from pathlib import Path
from datetime import datetime
from openai import OpenAI

# Set the base path
base_path = Path("../")  # One level up from the current working directory

# Add the src/ directory to sys.path using base_path
sys.path.append(str((base_path / "src").resolve()))

from tools.search_gutenberg_books import (
    SEARCH_GUTENBERG_BOOKS_TOOL,
    search_gutenberg_books
)
from tool_calling import run_tool_call_loop

In [11]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=getenv("OPENROUTER_API_KEY")
)

# Messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What are the titles of some Charles Dickens books?"}
]

# Tools + Mapping
tools = [
    SEARCH_GUTENBERG_BOOKS_TOOL
]

TOOL_MAPPING = {
    "search_gutenberg_books": search_gutenberg_books
}

MODEL = "gpt-4o-mini"

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # To allow for multiple runs

In [12]:
response, messages = run_tool_call_loop(
    openai_client=client,
    model=MODEL,  # or any tool-calling-capable model
    tools=tools,
    messages=messages,
    tool_mapping=TOOL_MAPPING
)


--- Iteration 1 ---
🛠️  Model requested tool: search_gutenberg_books with args: {'search_terms': ['Charles Dickens']}

--- Iteration 2 ---
🔚 Final model output:
 Here are some titles of books by Charles Dickens:

1. A Tale of Two Cities
2. Great Expectations
3. A Christmas Carol in Prose; Being a Ghost Story of Christmas
4. Oliver Twist
5. Hard Times


## Store the results

In [13]:
# Create timestamped directory
results_dir = base_path / "results" / timestamp / MODEL
results_dir.mkdir(parents=True, exist_ok=True)

# File paths (no timestamp in file names)
txt_path = results_dir / "final_response.txt"
json_path = results_dir / "message_history.json"

# Write the final response (text)
if response:
    txt_path.write_text(response, encoding="utf-8")

# Write the full message history (JSON)
with json_path.open("w", encoding="utf-8") as f:
    json.dump(messages, f, ensure_ascii=False, indent=2)

print(f"✅ Final response saved to: {txt_path}")
print(f"✅ Message history saved to: {json_path}")

✅ Final response saved to: ../results/2025-05-16_21-47-19/gpt-4o-mini/final_response.txt
✅ Message history saved to: ../results/2025-05-16_21-47-19/gpt-4o-mini/message_history.json
